In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

llm.invoke("국립공주대학교에 대해 설명해줘.")

AIMessage(content='국립공주대학교(國立公州大學校, Kongju National University)는 대한민국 충청남도 공주시에 위치한 국립대학교입니다. 1978년에 설립된 이 대학은 교육, 연구, 지역사회 발전에 기여하는 것을 목표로 하고 있습니다. \n\n국립공주대학교는 다양한 학부와 대학원 프로그램을 제공하며, 인문사회과학, 자연과학, 공학, 예술 등 여러 분야에서 교육을 실시하고 있습니다. 또한, 학생들에게 실무 경험을 쌓을 수 있는 기회를 제공하기 위해 다양한 산학협력 프로그램과 인턴십 기회를 마련하고 있습니다.\n\n대학 캠퍼스는 아름다운 자연환경 속에 위치해 있으며, 학생들이 학업과 여가를 즐길 수 있는 다양한 시설과 공간이 마련되어 있습니다. 또한, 국제 교류 프로그램을 통해 해외 대학과의 협력도 활발히 진행하고 있습니다.\n\n국립공주대학교는 지역사회와의 연계를 중요시하며, 지역 발전을 위한 다양한 프로젝트와 연구를 수행하고 있습니다. 이러한 노력은 학생들에게 지역 사회에 대한 이해와 책임감을 심어주는 데 기여하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 248, 'prompt_tokens': 18, 'total_tokens': 266, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BevpnwGSewXdgeS

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

hf_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\캡스톤디자인\chatbot-api\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain.text_splitter import CharacterTextSplitter

# 텍스트 분리
text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages

# 프롬프트 템플릿
system_message = """
당신은 공주대학교와 관련된 정보를 안내하는 AI입니다.
공식 문서나 공주대학교 사이트에서 제공되는 정보만 바탕으로 대답하세요.
문맥에서 명확한 정보가 없으면 "찾을 수 없습니다"라고 말해주세요.
정확한 출처를 아래와 같이 반드시 포함하세요.
파일명 :
부서/학과 :
URL :
"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("placeholder", "{memory}"),
        ("user", "🔍 검색된 문서:\n{context}"),
        ("human", "{input}"),
    ]
)

# 출력 파서
parser = StrOutputParser()

# 트리머 설정
trimmer = trim_messages(
    max_tokens=500,
    token_counter=llm,
    strategy="last",
    include_system=True,
    start_on="human"
)

In [6]:
from langchain_community.chat_message_histories.sql import SQLChatMessageHistory
import os

history_store: dict[str, SQLChatMessageHistory] = {}

def init_session(session_id: str):
    if session_id not in history_store:
        mysql_url = os.getenv("DATABASE_URL")
        history_store[session_id] = SQLChatMessageHistory(
            connection=mysql_url,
            table_name="chat_history",
            session_id=session_id,
            session_id_field_name="session_id",
        )
        history_store[session_id].add_message(SystemMessage(content=system_message.strip()))

def get_session_history(session_id: str):
    return history_store[session_id]

In [7]:
init_session("session_1")
get_session_history("session_1")

In [8]:
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.documents import Document
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

DEPARTMENT_MAP = {
    "ALL(전체)" : "knu_chroma_db_all",
    "Software Department (소프트웨어학과)": "knu_chroma_db_software",
    "Department of Computer Engineering (컴퓨터공학과)": "knu_chroma_db_computer",
    "Department of Architecture (건축학과)": "knu_chroma_db_architecture",
    "Department of Chemical Engineering (화학공학부)/Chemical Engineering Major (화학공학전공)": "knu_chroma_db_chemical_engineering_major",
    "National Kongju University SW Centered University Business Group (국립공주대학교 SW중심대학사업단)": "knu_chroma_db_sw_centerd_university_business_group",
}

retriever = None
rag_chain = None

def rebuild_chain(selected_dept: str = None):
    global retriever, rag_chain

    # 사용자가 부서를 선택한 경우: 해당 부서 전용 DB만 불러와 retriever 구성(빠름)
    if selected_dept:
        db_path = f"./ChromaDB/{DEPARTMENT_MAP[selected_dept]}"
        db = Chroma(persist_directory=db_path, embedding_function=hf_embeddings)
    else:
        # 부서를 선택하지 않은 경우: 모든 부서의 문서가 저장된 DB를 로드하여 retriever 구성(느림)
        db_path = f"./ChromaDB/knu_chroma_db"
        db = Chroma(persist_directory=db_path, embedding_function=hf_embeddings)
    print(f"DB 로딩 완료: {db_path}")

    """
    벡터 거리 기반 의미 검색
    질문 -> MultiQuery(3개의 서브 질문)
    서브 질문 1 -> ChromaDB에서 mmr방식(다양하고도 유사한 문서를 찾는 방식)으로 검색하여 10개 후보 중 5개를 선택
    서브 질문 2 -> ChromaDB에서 mmr방식으로 검색하여 10개 후보 중 5개를 선택
    서브 질문 3 -> ChromaDB에서 mmr방식으로 검색하여 10개 후보 중 5개를 선택
    -> 전체 15개 청크 검색됨 -> 중복 제거 -> 고유 청크 N개(15개 이하)

    키워드 기반 재정렬 - Document 리스트 상태에서 작동함
    청크 N개를 대상으로 BM25(키워드(=질문에서 나온 단어)가 문서 안에 얼마나 잘 등장하는지 관련성을 수치화하는 방식)
    -> top 3개 선택
    -> LLM context에 top3 청크 3개 삽입 -> 답변 생성
    """
    # 전체 문서 로딩 후 BM25 구성용 문서 리스트 확보
    results = db.get(include=["documents", "metadatas"])
    docs = [Document(page_content=doc, metadata=meta) for doc, meta in zip(results["documents"], results["metadatas"])]
    print(docs[:3])  # 로딩된 문서 일부 출력

    # Chroma + BM25 혼합 검색기 구성
    chroma_retriever = db.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 3, "fetch_k": 10, "lambda_mult": 0.5}
    )

    bm25_retriever = BM25Retriever.from_documents(docs)
    bm25_retriever.k = 3
    base_retriever = EnsembleRetriever(retrievers=[chroma_retriever, bm25_retriever])
    retriever = MultiQueryRetriever.from_llm(base_retriever, llm=llm)

    # 최종 RAG 체인
    rag_chain_core = {
        "memory": trimmer,      # trimmer 적용
        "context": retriever,
        "input": RunnablePassthrough()
    } | prompt_template | llm | parser

    # 최종 RAG 체인
    rag_chain = RunnableWithMessageHistory(
        rag_chain_core,
        get_session_history,
        input_messages_key="input",
        history_messages_key="memory"
    )

# 질문 처리
def answer_question(question: str, session_id: str) -> str:
    if not session_id:
        return "⚠️ 세션을 먼저 생성하거나 선택해주세요."

    init_session(session_id)

    # rag_chain의 입력 형식에 맞춰 dict 구성
    chain_input = {
        "memory": None,        # RunnableWithMessageHistory가 알아서 처리
        "context": None,       # retriever가 알아서 처리
        "input": question
    }

    # RAG 체인 호출 → 답변 생성(자동으로 history에 기록됨)
    answer = rag_chain.invoke(
        chain_input,
        config={"configurable": {"session_id": session_id}}
    )
    return answer

In [9]:
rebuild_chain()

DB 로딩 완료: ./ChromaDB/knu_chroma_db
[Document(metadata={'department': 'Software Department (소프트웨어학과)', 'file_name': '학문의세계와직업, K-Value 미래설계Ⅰ, K-Value 미래설계 Ⅱ 미이수 학생 대체 과목 안내.md', 'source': '/content/drive/MyDrive/Cheonan Campus All Departments (천안캠퍼스 모든 학과)/Software Department (소프트웨어학과)/커뮤니티 (규정자료실)/학문의세계와직업, K-Value 미래설계Ⅰ, K-Value 미래설계 Ⅱ 미이수 학생 대체 과목 안내.md', 'url': 'https://sw.kongju.ac.kr/bbs/ZD1180/1426/407683/artclView.do'}, page_content='# 학문의세계와직업, K-Value 미래설계Ⅰ, K-Value 미래설계 Ⅱ 미이수 학생 대체 과목 안내\n\n**출처:** [https://sw.kongju.ac.kr/bbs/ZD1180/1426/407683/artclView.do](https://sw.kongju.ac.kr/bbs/ZD1180/1426/407683/artclView.do)\n**작성자:** 임성철\n**작성일:** 2025.03.31\n\n## 본문\n학문의세계와직업, K-Value 미래설계Ⅰ, K-Value 미래설계 Ⅱ 미이수 학생 대체 과목 안내'), Document(metadata={'source': '/content/drive/MyDrive/Cheonan Campus All Departments (천안캠퍼스 모든 학과)/Software Department (소프트웨어학과)/커뮤니티 (규정자료실)/학문의세계와직업, 

In [10]:
answer_question("국립공주대학교의 소프트웨어학과에 대해 알려줘.", "session_1")

'국립공주대학교의 소프트웨어학과는 컴퓨터소프트웨어 인력 양성을 위해 다양한 소프트웨어 교과목을 제공하고 있습니다. 인공지능, 사물인터넷, ERP, 모바일, 웹, 게임, 서버 관리 등 여러 응용 분야에 진출할 수 있도록 이론과 실습을 병행하여 교육하고 있습니다. 학생들은 입학 후 다양한 프로그래밍 언어를 배우며 코딩 능력을 향상시키는 기회를 가집니다.\n\n자세한 내용은 아래의 링크를 통해 확인하실 수 있습니다.\n\n파일명 : 소프트웨어학과 홈페이지  \n부서/학과 : 소프트웨어학과  \nURL : [https://sw.kongju.ac.kr/ZD1180/11621/subview.do](https://sw.kongju.ac.kr/ZD1180/11621/subview.do)'

In [11]:
answer_question("국립공주대학교의 소프트웨어학과의 장학 조건에에 대해 알려줘.", "session_1")

'국립공주대학교 소프트웨어학과의 장학 조건에 대한 정보는 다음과 같습니다.\n\n소프트웨어전공 장학생 선발 규정에 따르면, 장학생은 당해 학기 교과목 성적과 가산점을 합산하여 상위 점수 자 순으로 선발됩니다. 또한, 1회 이상 지도교수의 상담을 수행한 학생에 한하여 장학금 수혜가 가능합니다. 성적은 평점평균(4.5 만점)을 기준으로 하며, 가산점은 최대 0.5점까지 부여됩니다.\n\n자세한 내용은 아래의 링크를 통해 확인하실 수 있습니다.\n\n파일명 : 소프트웨어전공 장학생 선발 규정  \n부서/학과 : 소프트웨어학과  \nURL : [https://sw.kongju.ac.kr/bbs/ZD1180/1426/208959/artclView.do](https://sw.kongju.ac.kr/bbs/ZD1180/1426/208959/artclView.do)'

In [12]:
answer_question("내가 이전에 뭐라고 질문했어?", "session_1")

'당신은 국립공주대학교의 소프트웨어학과에 대해 질문하셨고, 그에 대한 장학 조건에 대해서도 문의하셨습니다. \n\n1. 소프트웨어학과에 대한 정보:\n   - 국립공주대학교의 소프트웨어학과는 다양한 소프트웨어 교과목을 제공하며, 인공지능, 사물인터넷, ERP, 모바일, 웹, 게임, 서버 관리 등 여러 분야에 진출할 수 있도록 교육하고 있습니다.\n   - URL: [소프트웨어학과 홈페이지](https://sw.kongju.ac.kr/ZD1180/11621/subview.do)\n\n2. 장학 조건에 대한 정보:\n   - 소프트웨어전공 장학생은 당해 학기 교과목 성적과 가산점을 합산하여 상위 점수 순으로 선발되며, 1회 이상 지도교수의 상담을 수행한 학생에 한하여 장학금 수혜가 가능합니다.\n   - URL: [소프트웨어전공 장학생 선발 규정](https://sw.kongju.ac.kr/bbs/ZD1180/1426/208959/artclView.do)'